# Assignment 1. Opinion Polling

In this assignment, you will be expected to analyze a dataset on your own and answer questions about your findings. 

You’ve been hired as a consultant to predict how a state school board election will turn out.
* There are three candidates and all voters must vote for one of them: Pearle Goodman, Masako Holley, Genevieve Gallegos.
* The candidate with the final highest vote count wins the election.
* You are given the list of registered voters here:
https://github.com/sjyk/cmsc21800/blob/master/voters.csv
* The state gives you two samples of data one collected by SurveyMonkey and one collected by Qualtrics:
https://github.com/sjyk/cmsc21800/blob/master/survey_monkey.csv 
https://github.com/sjyk/cmsc21800/blob/master/qualtrics.csv

## Initial Steps
Let's first get our data analysis environment setup by loading all of the datasets:

In [33]:
import pandas as pd

voter_roll = pd.read_csv('voters.csv') #reads the data as a csv file

In [34]:
survey_monkey = pd.read_csv('survey_monkey.csv')
qualtrics = pd.read_csv('qualtrics.csv')

survey_monkey[:10]

,Voter,Vote
0,Robert Wiltse,Genevieve Gallegos
1,Ellen Delrio,Pearle Goodman
2,Armando Dawson,Genevieve Gallegos
3,Sarah Ybarra,Masako Holley
4,Roger Taylor,Genevieve Gallegos
5,Ricky Applewhite,Genevieve Gallegos
6,Rufus Harrison,Genevieve Gallegos
7,Emmitt Engelking,Genevieve Gallegos
8,Michael Curry,Masako Holley
9,Elmer Jones,Genevieve Gallegos


In [40]:
candidates = set(survey_monkey['Vote']).union(set(qualtrics['Vote']))
candidates

{'Genevieve Gallegos', 'Masako Holley', 'Pearle Goodman'}

In [14]:
#How do you get the fraction for each candidate in survey monkey?

,Voter
Vote,
Genevieve Gallegos,0.59
Masako Holley,0.35
Pearle Goodman,0.06


In [15]:
#How do you get the fraction for each candidate in qualtrics

,Voter
Vote,
Genevieve Gallegos,0.42
Masako Holley,0.50
Pearle Goodman,0.08


In [21]:
#How do you link the survey data to the voter rolls?

,Voter,Vote,Gender,Age,County
0,Michael Thomas,Masako Holley,male,36-45,Mountain Farm
1,Jeri Cowley,Genevieve Gallegos,female,36-45,Mountain Farm
2,Alexander Moose,Genevieve Gallegos,male,36-45,Mountain Farm
3,Linda Abraham,Masako Holley,female,36-45,Mountain Farm
4,Alfred Blocker,Genevieve Gallegos,male,36-45,Mountain Farm
5,Leona Hill,Masako Holley,female,65+,Black
6,Brittany Silva,Masako Holley,female,46-55,Mountain Farm
7,Lawrence Knight,Masako Holley,male,46-55,Mountain Farm
8,Michael White,Genevieve Gallegos,male,56-65,Mountain Farm
9,Annette Ealy,Masako Holley,female,36-45,Bailey


In [23]:
Nqualtrics = len(qualtrics)
Nsurvey = len(survey_monkey)

Nqualtrics, Nsurvey

(50, 100)

### Q1. The SurveyMonkey data shows Genevieve Gallegos winning 59% vote of 100 people polled and the Qualtrics data shows her losing with 42% vote of 50 people polled.  Which of the following best describes the likelihood that a difference this large (>17%) happened purely by random chance and not an error in the polling process?

In [24]:
"""
Let's calculate the likelihood that a single poll could be off by 17%
"""

import numpy as np

#how do you calculate a 95% confidence interval?

50 polled:  {'68% +/-': 0.07071067811865475, '95% +/-': 0.13859292911256332, '99% +/-': 0.1817264427649427}
100 polled:  {'68% +/-': 0.05, '95% +/-': 0.098, '99% +/-': 0.1285}


Clearly, it is very unlikely. So (c) is the right answer.

### Q2. The data provider suspects that the SurveyMonkey dataset is biased. What do you think?

In [26]:
#How would you find out if the data is biased or not?

Clearly it looks like the dataset is gender-biased. Let's see if this could have happened by chance.

In [ ]:
#How likely is this by chance?

### Q3. Which of the following best describes the margin of error of the Qualtrics poll

In [28]:
#?

### Q4.  A news report suggests that Pearle Goodman is dropping out of the election. Is it clear which candidate benefits from her departure?

In [29]:
#how would you figure this out?

As you can see above Pearle Goodman has the same male-female break down as Genevieve Gallagos. So it would be reasonable to assume her votes would got to her. However, we also accepted arguments that the sample size was too small to tell.